In [2]:
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader

In [3]:
import pandas as pd
iris = pd.read_csv('https://raw.githubusercontent.com/mwaskom/seaborn-data/master/iris.csv')

In [4]:
data = torch.tensor( iris[iris.columns[0:4]].values ).float()
labels = torch.zeros(len(data), dtype=torch.long)
labels[iris.species=='versicolor'] = 1
labels[iris.species=='virginica']  = 2

In [5]:
train_data,test_data, train_labels,test_labels = train_test_split(data, labels, test_size=.2)
train_dataDataset = torch.utils.data.TensorDataset(train_data,train_labels)
test_dataDataset  = torch.utils.data.TensorDataset(test_data,test_labels)
train_loader = DataLoader(train_dataDataset,batch_size=64, shuffle=True, drop_last=True)
test_loader  = DataLoader(test_dataDataset,batch_size=test_dataDataset.tensors[0].shape[0])

In [6]:
def createANewModel(L2lambda):
  ANNiris = nn.Sequential(
      nn.Linear(4,64),
      nn.ReLU(),
      nn.Linear(64,64),
      nn.ReLU(),
      nn.Linear(64,3),
        )
  lossfun = nn.CrossEntropyLoss()
  optimizer = torch.optim.SGD(ANNiris.parameters(),lr=.005,weight_decay=L2lambda)
  return ANNiris,lossfun,optimizer

In [7]:
numepochs = 1000
def trainTheModel():
  trainAcc = []
  testAcc  = []
  losses   = []

In [11]:
numepochs = 1000
def trainTheModel():

  trainAcc = []
  testAcc  = []
  losses   = []

  for epochi in range(numepochs):
    batchAcc  = []
    batchLoss = []

    for X,y in train_loader:
      yHat = ANNiris(X)
      loss = lossfun(yHat,y)
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()

      batchAcc.append( 100*torch.mean((torch.argmax(yHat,axis=1) == y).float()).item() )
      batchLoss.append( loss.item() )

    trainAcc.append( np.mean(batchAcc) )
    losses.append( np.mean(batchLoss) )

    ANNiris.eval()
    X,y = next(iter(test_loader))
    predlabels = torch.argmax( ANNiris(X),axis=1 )

    testAcc.append( 100*torch.mean((predlabels == y).float()).item() )
    ANNiris.train()
  return trainAcc,testAcc,losses


In [12]:
L2lambda = .01
ANNiris,lossfun,optimizer = createANewModel(L2lambda)

# train the model
trainAcc,testAcc,losses = trainTheModel()